In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
import os

import numpy as np

In [12]:
import zipfile
import os
import shutil


shutil.rmtree("flowers", ignore_errors=True)


zip_path = "flower-classes-5.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("flowers")

print(os.listdir("flowers"))


['flower_photos']


In [14]:
subdirs = os.listdir("flowers")
if len(subdirs) == 1 and os.path.isdir(os.path.join("flowers", subdirs[0])):
    nested_dir = os.path.join("flowers", subdirs[0])
    for item in os.listdir(nested_dir):
        shutil.move(os.path.join(nested_dir, item), "flowers")
    shutil.rmtree(nested_dir)


print("Final folders inside 'flowers':", os.listdir("flowers"))


Final folders inside 'flowers': ['.ipynb_checkpoints', 'daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [16]:
import shutil
shutil.rmtree("flowers/.ipynb_checkpoints", ignore_errors=True)

In [18]:
IMG_SIZE = 160
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    "flowers",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    "flowers",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu',
                 input_shape=(160, 160, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(5, activation='softmax'))

model.summary()


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 160, 160, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 80, 80, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 80, 80, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 40, 40, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 40, 40, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 20, 20, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 20, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 500)                 │      12,800,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           2,505 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,826,589 (48.93 MB)

 Trainable params: 12,826,589 (48.93 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.3123 - loss: 1.7240 - val_accuracy: 0.4966 - val_loss: 1.1452
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 539ms/step - accuracy: 0.5539 - loss: 1.0935 - val_accuracy: 0.6033 - val_loss: 1.0798
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 49s 533ms/step - accuracy: 0.6646 - loss: 0.8706 - val_accuracy: 0.6170 - val_loss: 0.9322
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 548ms/step - accuracy: 0.7120 - loss: 0.7336 - val_accuracy: 0.6320 - val_loss: 0.9336
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 544ms/step - accuracy: 0.7782 - loss: 0.5918 - val_accuracy: 0.6539 - val_loss: 0.9273
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 547ms/step - accuracy: 0.8518 - loss: 0.4366 - val_accuracy: 0.6457 - val_loss: 1.0472
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 537ms/step - accuracy: 0.9102 - loss: 0.2694 - val_accuracy: 0.6689 - val_loss: 1.0740
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 553ms/step - accuracy: 0.9403 - loss: 0.2006 - val_accura

In [28]:
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img('dandelion.jpg', target_size=(160, 160))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
class_names = list(train_data.class_indices.keys())
print("the new image is:", class_names[np.argmax(predictions)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
the new image is: dandelion
